### Question 2 - Top 3 Cities

  For each country, compute the top 3 cities with best air quality and the top 3 cities with poorest air quality, updated weekly, i.e., averaged over a week (7 days).

 **Requirement**: Solve this question using Spark Core, Spark Dataframes and Spark SQL.

In [1]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive') # Faz o mount da drive

Mounted at /content/drive


In [2]:
#@title Install Pyspark
!pip install --quiet pyspark # Faz a instalação do Pyspark

     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 52.6 MB/s 


In [4]:
#@title Dataset
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-0*.csv > files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-1*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-2*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-3*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-02-0*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-02-1*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-02-2*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-0*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-1*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-2*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-3*.csv >> files

In [5]:
#@title Resolution using Spark Core
import pyspark
import json
import string
from datetime import datetime
from operator import *
from pyspark import pandas

psc = pyspark.SparkContext('local[*]') # Cria o SparkContext, em local mode

try:

  data1 = psc.textFile('/content/files') # Faz o carregamento dos dados
  data11 = data1.map(lambda line: line.split(';')).filter( lambda values: len(values) == 7).filter( lambda values: len(values[5])>0).filter( lambda values: len(values[6])>0)
  # Faz o split dos campos de cada linha, por ;, o filtro de cada linha por forma a ficar apenas com aquelas que têm 7 campos e o filtro das linhas que contêm valores positivos para os valores de p1 e p2
  data12 = data11.map( lambda v: (int(v[0]), (v[0], datetime.strptime(v[4],'%Y-%m-%dT%H:%M:%S').isocalendar()[1], float(v[5]), float(v[6]))))
  # Faz o map, em que a key corresponde ao sensor_id (integer) e a value à tuple (sensor_id, semana do ano, float(p1), float(p2))
  data2 = psc.textFile('/content/drive/MyDrive/projeto_spbd/sensors_all.json') # Faz o carregamento do ficheiro JSON
  data21 = data2.map(lambda line: json.loads(line)) # Faz o map do ficheiro JSON
  data22 = data21.flatMap(lambda line: line) # Faz o flatMap do data21, onde cada linha corresponde a um dicionário com os valores correspondentes às variáveis do sensor_id, country, city e country_code (todas elas string)
  data23 = data22.map(lambda x: (int(x["sensor_id"]), (x["country"], x["city"]))) # Faz o map de data22, onde a key corresponde ao sensor_id (integer) e a value à tuple (country, city)

  sensors = data23.join(data12) # Faz o joint dos dois RDDs, data23 e data12, pela key sensor_id
  sensors2 = sensors.map(lambda x: ((x[1][0][0],x[1][0][1],x[1][1][1]), (x[1][1][2], x[1][1][3], 1))) # Faz o map, em que a key corresponde à tuple (country, city, semana) e value à tuple (p1, p2, 1)
  sensors3 = sensors2.reduceByKey( lambda a, b: (a[0]+b[0], a[1]+b[1], a[2]+b[2])) # Faz o reduceByKey, onde é feita a operação: (p1+p1, p2+p2, 1+1), para cada linha
  sensors4 = sensors3.map( lambda x: ((x[0][0],x[0][2]), ((x[1][0]+x[1][1])/x[1][2], x[0][1]))).sortBy(lambda x: (x[0][0], x[0][1], x[1][0])).map(lambda x: (x[0],[x[1]]))
  # Faz o map, em que a key corresponde à tuple (country, semana) e a value à tuple (media de p1+p2, city). É feito o sortBy por country, semana e media de p1+p2 (métrica). É feito o map, em que a key corresponde à tuple (country, semana) e a value a uma LISTA com a tuple (media de p1+p2, city).
  sensors5 = sensors4.reduceByKey( lambda a, b: a + b).sortByKey( lambda x: x[0]) # É feito o reduceByKey, onde a key é a tuple (country, semana) e a value a LISTA com as VÁRIAS tuples (métrica, city). É feito o sort pela key. 

  for s, v in sensors5.collect():
    print(s, v[-3:], v[0:3])
  # É feito o print da key (country, semana) e dos 3 melhores métricas (e correspondente cidade) e 3 piores métricas (e correspondente cidade)
  psc.stop()
except Exception as e: 
  print(e)
  psc.stop()

('France', 1) [(340.6875, 'Tullins'), (886.99, 'Gallardon'), (2097.0088888888886, 'Luitré')] [(0.3333333333333334, 'Arnouville'), (0.9644444444444444, 'Sargé-lès-le-Mans'), (1.6175000000000002, 'Brest')]
('France', 2) [(144.1007142857143, 'Crusnes'), (160.1616666666667, 'Fonsorbes'), (2999.8, 'Luitré')] [(0.11454545454545456, 'Arnouville'), (1.093076923076923, 'Sargé-lès-le-Mans'), (2.6325000000000003, 'Brest')]
('France', 3) [(103.99766666666666, 'Anglet'), (238.21727272727276, 'Péchabou'), (2999.8, 'Luitré')] [(0.27166666666666667, 'Arnouville'), (0.55, 'Brest'), (1.1764285714285714, 'Sargé-lès-le-Mans')]
('France', 4) [(180.86461538461538, 'Passy'), (276.13205882352946, 'Serémange-Erzange'), (759.2719999999999, 'Luitré')] [(0.6133333333333335, 'Arnouville'), (1.2546153846153847, 'Sargé-lès-le-Mans'), (2.631666666666667, 'Figeac')]
('France', 5) [(172.39142857142858, 'Luitré'), (234.9975, 'Bois-Colombes'), (339.0488888888889, 'Tinténiac')] [(0.16363636363636364, 'Arnouville'), (0.304